In [ ]:
pip install opencv-python numpy scikit-learn joblib

In [ ]:
import cv2
import numpy as np
import time
import glob
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score
from joblib import Parallel, delayed
import os

TRAIN_IMG_DIR = "./ADEChallengeData2016/images/training/"
TRAIN_ANN_DIR = "./ADEChallengeData2016/annotations/training/"
VAL_IMG_DIR   = "./ADEChallengeData2016/images/validation/"
VAL_ANN_DIR   = "./ADEChallengeData2016/annotations/validation/"

FLOOR_ID = 4
TRAIN_SAMPLES = 100000
VAL_SAMPLES   = 20000
N_JOBS = -1

def process_image(img_path, ann_path, samples_per_image):
    """
    Reads an image and annotation, and returns a sampled subset of features and labels.
    """
    try:
        img = cv2.imread(img_path)
        ann = cv2.imread(ann_path, 0)
        
        if img is None or ann is None:
            return None, None

        H, W, _ = img.shape
        mask = (ann == FLOOR_ID).astype(np.int32)

        rgb = img.reshape(-1, 3).astype(np.float32) / 255.0
        labels = mask.reshape(-1)

        xs, ys = np.meshgrid(np.arange(W), np.arange(H))
        xs = xs.reshape(-1, 1) / W
        ys = ys.reshape(-1, 1) / H
        
        features = np.hstack([rgb, xs, ys])

        if len(labels) > samples_per_image:
            idx = np.random.choice(len(labels), samples_per_image, replace=False)
            return features[idx], labels[idx]
        else:
            return features, labels
            
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        return None, None

def load_data_parallel(image_paths, ann_paths, total_samples):
    n_images = len(image_paths)
    if n_images == 0: return np.array([]), np.array([])
    
    samples_per_image = max(1, total_samples // n_images)
    
    print(f"Processing {n_images} images with {N_JOBS} jobs...")
    results = Parallel(n_jobs=N_JOBS)(delayed(process_image)(img, ann, samples_per_image) 
                                      for img, ann in zip(image_paths, ann_paths))
    
    X_list = [res[0] for res in results if res[0] is not None]
    y_list = [res[1] for res in results if res[1] is not None]
    
    if not X_list:
        return np.array([]), np.array([])
        
    X_all = np.vstack(X_list)
    y_all = np.hstack(y_list)
    
    idx = np.random.permutation(len(X_all))
    
    if len(idx) > total_samples:
        idx = idx[:total_samples]
        
    return X_all[idx], y_all[idx]


train_images = sorted(glob.glob(TRAIN_IMG_DIR + "*.jpg"))[:50]
train_annots = sorted(glob.glob(TRAIN_ANN_DIR + "*.png"))[:50]

print("Loading Training Data...")
start_load = time.time()
X_train, y_train = load_data_parallel(train_images, train_annots, TRAIN_SAMPLES)
print(f"Loaded {len(X_train)} samples in {time.time() - start_load:.2f}s")

val_images = sorted(glob.glob(VAL_IMG_DIR + "*.jpg"))[:20]
val_annots = sorted(glob.glob(VAL_ANN_DIR + "*.png"))[:20]

print("Loading Validation Data...")
X_val, y_val = load_data_parallel(val_images, val_annots, VAL_SAMPLES)

# Train SVM
print("Training SVM...")
svm = LinearSVC(C=1.0, max_iter=3000)

start_train = time.time()
svm.fit(X_train, y_train)
train_time = time.time() - start_train

# Inference
print("Running Inference...")
start_test = time.time()
y_pred = svm.predict(X_val)
test_time = time.time() - start_test

accuracy  = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred, zero_division=0)
recall    = recall_score(y_val, y_pred, zero_division=0)

print("\n=== Optimized Method 2: RGB + (x,y), LinearSVC ===")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"Training time : {train_time:.4f}s")
print(f"Inference time: {test_time:.4f}s")


Loading Training Data...
Processing 50 images with -1 jobs...
Loaded 100000 samples in 0.75s
Loading Validation Data...
Processing 20 images with -1 jobs...
Training SVM...
Running Inference...

=== Optimized Method 2: RGB + (x,y), LinearSVC ===
Accuracy : 0.9350
Precision: 0.0536
Recall   : 0.0024
Training time : 0.0393s
Inference time: 0.0005s
